## 142: Agregar Roles de Usuario y Restringir Acceso por Permisos

Hoy aprenderás a añadir roles de usuario (como "admin" o "usuario") y a restringir rutas específicas para que solo ciertos roles puedan acceder a determinadas funciones (por ejemplo, enviar reportes solo si eres admin).

Esto permite:

✅ Control de privilegios

🛡️ Protección de funciones críticas

👥 Soporte para múltiples tipos de usuarios

🧩 1. Modificar la función de login para incluir el rol
En auth.py:

python
Copiar
Editar
def crear_token(data: dict, expires_delta: timedelta = None):
    to_encode = data.copy()
    expire = datetime.utcnow() + (expires_delta or timedelta(minutes=EXPIRE_MINUTES))
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
En main.py, actualiza el login para incluir el rol:

python
Copiar
Editar
USUARIO = {
    "username": "admin",
    "password": "1234",
    "email": "admin@correo.com",
    "rol": "admin"
}

@app.post("/login")
def login(form_data: OAuth2PasswordRequestForm = Depends()):
    if form_data.username != USUARIO["username"] or form_data.password != USUARIO["password"]:
        raise HTTPException(status_code=401, detail="Credenciales incorrectas")
    
    token = crear_token({
        "sub": form_data.username,
        "rol": USUARIO["rol"]
    })
    return {"access_token": token, "token_type": "bearer"}
🧩 2. Crear una función para validar que el usuario tenga rol "admin"
En auth.py:

python
Copiar
Editar
def verificar_admin(token: str = Depends(oauth2_scheme)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        if payload.get("rol") != "admin":
            raise HTTPException(status_code=403, detail="Acceso solo para administradores")
        return payload
    except JWTError:
        raise HTTPException(status_code=403, detail="Token inválido o expirado")
🧩 3. Usar esta validación en una ruta protegida
En main.py, cambia:

python
Copiar
Editar
@app.post("/enviar-reporte/")
def enviar_reporte(data: ReporteRequest, user: dict = Depends(verificar_admin)):
    ...
Esto solo permitirá el acceso a usuarios con rol = "admin".

🧪 4. Probar el sistema
Login como admin:
bash
Copiar
Editar
curl -X POST http://localhost:8000/login \
-F 'username=admin' -F 'password=1234'
Luego usa el token en:

bash
Copiar
Editar
curl -X POST http://localhost:8000/enviar-reporte/ \
-H "Authorization: Bearer TU_TOKEN_JWT" \
-H "Content-Type: application/json" \
-d '{"email":"alguien@correo.com", "data":[{"Fecha":"2024-01-01","Tipo":"PDF","Categoría":"Anual"}]}'
Si no eres admin, verás:

json
Copiar
Editar
{
  "detail": "Acceso solo para administradores"
}